In [ ]:
import os
import subprocess
from pathlib import Path
import concurrent.futures
import warnings
import pandas as pd
from biopandas.pdb import PandasPdb
from Bio.PDB import PDBParser, PDBIO
from Bio import PDB
from abnumber import Chain
import subprocess
import shutil
import os


warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

def run_command(cmd):
    process = subprocess.Popen(cmd, shell=True)
    process.wait()
    return process.returncode

software_foldx="software/foldx/foldx"
wt_folder= "example_3D/wt_data"
mut_folder="example_3D/mut_data"

In [ ]:
pdb_path=wt_folder+'/ms298r.pdb'
dst = wt_folder+'/relaxed.pdb'
shutil.copy(pdb_path, dst) # 复制命名文件

def get_chain_sequences(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure('PDB_structure', pdb_file)
    chain_sequences = {}
    for model in structure:
        for chain in model:
            chain_id = chain.get_id()
            sequence = ''
            for residue in chain:
                if PDB.is_aa(residue, standard=True):
                    sequence += PDB.Polypeptide.three_to_one(residue.get_resname())
            chain_sequences[chain_id] = sequence
    return chain_sequences

chain_sequences = get_chain_sequences(pdb_path)
for chain_id, sequence in chain_sequences.items():
    print(f"Chain {chain_id} :{sequence}")
    try:
        chain = Chain(sequence, scheme='imgt')
        if len(chain)>0:
            if chain.chain_type=='H':
                chain_H=sequence
            elif chain.chain_type=='L':
                chain_L=sequence
    except:
        pass

In [ ]:

pdb_seq=chain_H
print(pdb_seq)
chain_id='H'
mut_pos=45
mut_res='L'

mut_seq=pdb_seq[:mut_pos-1] +mut_res+pdb_seq[mut_pos:]
print(mut_seq)

data = pd.DataFrame()
data.loc['test', 'metric'] = 'ddGdTm'
data.loc['test', 'version'] = '3D'
data.loc['test', 'pdb_id'] = '0pdb_id'
data.loc['test', 'model_id'] = '1'
data.loc['test', 'chain_id'] = chain_id
data.loc['test', 'pdb_seq'] = pdb_seq
data.loc['test', 'seq'] = ''
data.loc['test', 'mut_pos'] = mut_pos
data.loc['test', 'mut_res'] = mut_res
data.loc['test', 'pH'] = 7
data.loc['test', 'temperature'] = 25
os.system('mkdir -p ./example_3D/wt_data')
os.system('mkdir -p ./example_3D/mut_data')
data.to_csv('./example_3D/mut_info.csv')

content=f">result\n{pdb_seq}"
with open(f'{wt_folder}/result.fasta', 'w') as f:
    f.write(content)

# write individual_list.txt file
with open(f'{mut_folder}/individual_list.txt', 'w') as f:
    f.write('{}{}{}{};'.format(pdb_seq[mut_pos+1], chain_id,mut_pos, mut_res))

content=f">result\n{mut_seq}"
with open(f'{mut_folder}/result.fasta', 'w') as f:
    f.write(content)

In [ ]:
# WT
!python generate_features/fixed_embedding.py --fasta_file example_3D/wt_data/result.fasta --saved_folder example_3D/wt_data
#生成 fixed_embedding.pt
!python generate_features/esm2_embedding.py --fasta_file example_3D/wt_data/result.fasta --saved_folder example_3D/wt_data
#生成 esm2.pt
for i in range(1,6):
    print(i)
    !python generate_features/esm1v_logits.py --model_index {i} --fasta_file example_3D/wt_data/result.fasta --saved_folder example_3D/wt_data
#生成 esm1v-1.pt esm1v-2.pt esm1v-3.pt esm1v-4.pt esm1v-5.pt
os.makedirs(f"{wt_folder}/foldx_tmp", exist_ok=True)
#FoldX
cmd = f"{software_foldx} --command=RepairPDB --pdb=relaxed.pdb --pdb-dir={wt_folder} --output-dir={wt_folder}/foldx_tmp"
run_command(cmd)
cmd = f"cp {wt_folder}/foldx_tmp/relaxed_Repair.pdb {wt_folder}/relaxed_repair.pdb"
run_command(cmd)

!python generate_features/coordinate.py --pdb_file {wt_folder}/relaxed_repair.pdb --saved_folder {wt_folder}
!python generate_features/pair.py --coordinate_file {wt_folder}/coordinate.pt --saved_folder {wt_folder}

In [ ]:
# Mut
!python generate_features/fixed_embedding.py --fasta_file {mut_folder}/result.fasta --saved_folder {mut_folder}
!python generate_features/esm2_embedding.py --fasta_file {mut_folder}/result.fasta --saved_folder {mut_folder}
os.makedirs(f"{mut_folder}/foldx_tmp", exist_ok=True)
cmd = f"{software_foldx} --command=BuildModel --pdb=relaxed_repair.pdb --pdb-dir={wt_folder} --mutant-file={mut_folder}/individual_list.txt --numberOfRuns=3 --output-dir={mut_folder}/foldx_tmp"
run_command(cmd)
cmd = f"cp {mut_folder}/foldx_tmp/relaxed_repair_1_2.pdb {mut_folder}/relaxed_repair.pdb"
run_command(cmd)
!python generate_features/coordinate.py --pdb_file {mut_folder}/relaxed_repair.pdb --saved_folder {mut_folder}
!python generate_features/pair.py --coordinate_file {mut_folder}/coordinate.pt --saved_folder {mut_folder}
folder="example_3D"
!python generate_features/ensemble_ddGdTm.py --saved_folder {folder}
!python model_ddG_3D/predict.py --ensemble_feature {folder}/ensemble.pt --saved_folder {folder}
!python model_dTm_3D/predict.py --ensemble_feature {folder}/ensemble.pt --saved_folder {folder}
#!python model_fitness_3D/predict.py --ensemble_feature {folder}/ensemble.pt --saved_folder {folder}


with open('example_3D/result_ddG.txt', 'r') as file:
    ddG_content = file.read()
with open('example_3D/result_ddG.txt', 'w') as file:
    file.write('{}{}{}{};'.format(pdb_seq[mut_pos+1], chain_id,mut_pos, mut_res) + ddG_content)
    
with open('example_3D/result_dTm.txt', 'r') as file:
    dTm_content = file.read()
with open('example_3D/result_dTm.txt', 'w') as file:
    file.write('{}{}{}{};'.format(pdb_seq[mut_pos+1], chain_id,mut_pos, mut_res) + dTm_content)

In [ ]:
with open('example_3D/result_ddG.txt', 'r') as file:
    content = file.read()
    # 将新内容写入文件
with open('example_3D/result_ddG.txt', 'w') as file:
    file.write('{}{}{}{};'.format(pdb_seq[mut_pos+1], chain_id,mut_pos, mut_res) + content)